In [1]:
import random

import numpy as np
import matplotlib.pyplot as plt

In [8]:
# 尝试加载数据集
# 数据集文件的格式，x，y,label
with open('testSet.txt','r') as f:
    for line in f:
        print(line)
        #print([data for data in line.strip().split(' ')])
        
#function
def load_data(filename):
    ''' dataset的格式，x，y，label
    '''
    dataset, labels = [], []
    with open(filename, 'r') as f:
        for line in f:
            x, y, label = [float(i) for i in line.strip().split()]
            dataset.append([x, y])
            labels.append(label)
    return dataset, labels
        

3.542485	1.977398	-1

3.018896	2.556416	-1

7.551510	-1.580030	1

2.114999	-0.004466	-1

8.127113	1.274372	1

7.108772	-0.986906	1

8.610639	2.046708	1

2.326297	0.265213	-1

3.634009	1.730537	-1

0.341367	-0.894998	-1

3.125951	0.293251	-1

2.123252	-0.783563	-1

0.887835	-2.797792	-1

7.139979	-2.329896	1

1.696414	-1.212496	-1

8.117032	0.623493	1

8.497162	-0.266649	1

4.658191	3.507396	-1

8.197181	1.545132	1

1.208047	0.213100	-1

1.928486	-0.321870	-1

2.175808	-0.014527	-1

7.886608	0.461755	1

3.223038	-0.552392	-1

3.628502	2.190585	-1

7.407860	-0.121961	1

7.286357	0.251077	1

2.301095	-0.533988	-1

-0.232542	-0.547690	-1

3.457096	-0.082216	-1

3.023938	-0.057392	-1

8.015003	0.885325	1

8.991748	0.923154	1

7.916831	-1.781735	1

7.616862	-0.217958	1

2.450939	0.744967	-1

7.270337	-2.507834	1

1.749721	-0.961902	-1

1.803111	-0.176349	-1

8.804461	3.044301	1

1.231257	-0.568573	-1

2.074915	1.410550	-1

-0.743036	-1.736103	-1

3.536555	3.964960	-1

8.410143	0.025606	1

7.

In [ ]:
def clip(alpha, L, H):
    ''' 控制alpha的值到L和H之间.
    '''
    if alpha < L:
        return L
    elif alpha > H:
        return H
    else:
        return alpha

def select_j(i, m):
    ''' 在m中随机选择除了i之外剩余的数
    '''
    l = list(range(m))
    seq = l[: i] + l[i+1:]
    return random.choice(seq)

In [40]:
#获取系数vector的函数
def get_w(alphas, all_x, labels):
    ''' 
    通过已知数据点和拉格朗日乘子获得分割超平面参数w
    alphas, 所有的alpha
    all_x, 所有的x向量，n行（数据个数）
    labels，标签
        
    '''
    alphas, all_x, labels = np.array(alphas), np.array(all_x), np.array(labels)
    n,m=all_x.shape;
    #得到的yx和x的维数是一样的
    yx = labels.reshape(-1, 1)*np.array.ones([1,n])*all_x
    w = np.dot(yx.T, alphas)

    return w.tolist()

## 测试
alphas=[1 ,2, 3, 4]
print('alphas=',alphas)
a=np.array(alphas)
print('a=',a)
b=np.array(alphas)
print('b=',b)
#matrix的输入很广，array也一样
c=np.matrix(alphas)
print('c=',c)
d=np.matrix(c);
print('d=',d)
e,f=c.shape
print(e)
g=a.shape
#注意这里两种shape是不一样的，有差别的
print(g)
g=a.reshape(-1,1)
print(g.shape)
print(g.T*np.ones([4]))
#两种array乘法的结果等价
print(g.T*np.ones([1,4]))

alphas= [1, 2, 3, 4]
a= [1 2 3 4]
b= [1 2 3 4]
c= [[1 2 3 4]]
d= [[1 2 3 4]]
1
(4,)
(4, 1)
[[ 1.  2.  3.  4.]]
[[ 1.  2.  3.  4.]]


In [45]:
# SMO的一些步骤的测试
a=[1 ,2, 3, 4]
a=np.array(a)
a=a.reshape(2,2)
print(a)
y=[2,4]
#下面两行对比了横着乘和竖着乘是不一样的
print(a*y)
print(a*np.array(y).reshape(-1,1))

[[1 2]
 [3 4]]
[[ 2  8]
 [ 6 16]]
[[ 2  4]
 [12 16]]


In [ ]:
# SMO优化函数，simple版本
def simple_smo(dataset, labels, C, max_iter):
    ''' 简化版SMO算法实现，未使用启发式方法对alpha对进行选择.

    :param dataset: 所有特征数据向量
    :param labels: 所有的数据标签
    :param C: 软间隔常数, 0 <= alpha_i <= C
    :param max_iter: 外层循环最大迭代次数
    '''
    dataset = np.array(dataset)
    m, n = dataset.shape
    labels = np.array(labels)

    # 初始化参数，alphas的维数和数据的个数一样
    alphas = np.zeros(m)
    b = 0
    it = 0

    def f(x):
        "SVM分类器函数 y = w^Tx + b"
        # Kernel function vector.
        x = np.matrix(x).T
        data = np.matrix(dataset)
        ks = data*x

        # Predictive value.
        wx = np.matrix(alphas*labels)*ks
        fx = wx + b
        # fx是一个1x1的矩阵，所以返回0，0
        return fx[0, 0]

    all_alphas, all_bs = [], []

    while it < max_iter:
        pair_changed = 0
        for i in range(m):
            a_i, x_i, y_i = alphas[i], dataset[i], labels[i]
            fx_i = f(x_i)
            E_i = fx_i - y_i

            j = select_j(i, m)
            a_j, x_j, y_j = alphas[j], dataset[j], labels[j]
            fx_j = f(x_j)
            E_j = fx_j - y_j

            K_ii, K_jj, K_ij = np.dot(x_i, x_i), np.dot(x_j, x_j), np.dot(x_i, x_j)
            eta = K_ii + K_jj - 2*K_ij
            if eta <= 0:
                print('WARNING  eta <= 0')
                continue

            # 获取更新的alpha对
            a_i_old, a_j_old = a_i, a_j
            a_j_new = a_j_old + y_j*(E_i - E_j)/eta

            # 对alpha进行修剪
            if y_i != y_j:
                L = max(0, a_j_old - a_i_old)
                H = min(C, C + a_j_old - a_i_old)
            else:
                L = max(0, a_i_old + a_j_old - C)
                H = min(C, a_j_old + a_i_old)

            a_j_new = clip(a_j_new, L, H)
            a_i_new = a_i_old + y_i*y_j*(a_j_old - a_j_new)

            if abs(a_j_new - a_j_old) < 0.00001:
                print('WARNING   alpha_j not moving enough')
                continue

            alphas[i], alphas[j] = a_i_new, a_j_new

            # 更新阈值b
            #import ipdb; ipdb.set_trace()
            b_i = -E_i - y_i*K_ii*(a_i_new - a_i_old) - y_j*K_ij*(a_j_new - a_j_old) + b
            b_j = -E_j - y_i*K_ij*(a_i_new - a_i_old) - y_j*K_jj*(a_j_new - a_j_old) + b

            if 0 < a_i_new < C:
                b = b_i
            elif 0 < a_j_new < C:
                b = b_j
            else:
                b = (b_i + b_j)/2

            all_alphas.append(alphas)
            all_bs.append(b)

            pair_changed += 1
            print('INFO   iteration:{}  i:{}  pair_changed:{}'.format(it, i, pair_changed))

        if pair_changed == 0:
            it += 1
        else:
            it = 0
        print('iteration number: {}'.format(it))

    return alphas, b